In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor

In [ ]:
def reset_random_seeds():
    np.random.seed(42)
    tf.random.set_seed(42)

reset_random_seeds()

df = pd.read_csv('/content/Beef_test4.csv')
X = df.drop(columns=['CH4Y'])
y = df['CH4Y']
for col in df.columns:
    df[col] = df[col].apply(lambda x: str(x).strip() if isinstance(x, str) else x)
categorical_features = ['RM']
numerical_features = X.columns.difference(categorical_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
params = {
    'iterations': 800,
    'learning_rate': 0.1,
    'depth': 3,
    'loss_function': 'RMSE',
    'verbose': 0
}
model = CatBoostRegressor(**params)
model.fit(X_train, y_train,cat_features=categorical_features)
y_pred = model.predict(X_test)

In [ ]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'R^2 Score: {r2}')
print(f'RMSE on test set: {np.sqrt(mean_squared_error(y_test, y_pred))}')
std_y_test = np.std(y_test)
std_y_test= np.std(y_test)
std_y_pred= np.std(y_pred)
if np.any(y_test == 0):
    raise ValueError("y_test contains zero(s), which would cause division by zero in RMSPE calculation.")
rmspe = np.sqrt(np.mean(((y_test - y_pred) / y_test) ** 2))
print(f'RMSPE on test set: {rmspe:.2f}')
rsr = rmspe / std_y_test
print(f'RSR (RMSPE / std of y_test): {rsr:.4f}')
mb = np.mean((y_test - y_pred))**2
print(f'MB: {mb}')
from scipy.stats import pearsonr
r, pearson_p_value = pearsonr(y_test, y_pred)
sb = (std_y_pred - r*std_y_test)**2
print(f'SB: {sb}')
def concordance_correlation_coefficient(y_true, y_pred):
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    covariance = np.mean((y_true - mean_true) * (y_pred - mean_pred))

    numerator = 2 * covariance
    denominator = var_true + var_pred + (mean_true - mean_pred) ** 2

    ccc = numerator / denominator
    return ccc

ccc = concordance_correlation_coefficient(y_test, y_pred)
print(f'CCC: {ccc:.4f}')

In [ ]:
model.save_model('/content/drive/MyDrive/Beef_cattle_CH4Ypredictiion.cbm')